This is a complete but still being tested workflow for generating irrigated cropping extents

Go to the user inputs section and input the required info, then run the rest of the cells.
Results will be in your results folder

### Libraries

In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
import dask

import rasterio.features
from osgeo import gdal, ogr
import os
from rsgislib.segmentation import segutils
from rasterstats import zonal_stats

#import custom functions
import sys
sys.path.append('src')
import DEAPlotting, SpatialTools, BandIndices
from load_data import load_data
from transform_tuple import transform_tuple
from imageSeg import imageSeg
from query_from_shp import query_from_shp

## User Inputs

In [ ]:
# where is the dcStats MaxNDVI tif and results folder?
data = 'data/datacube_stats/summer_2004_2018/tifs/ndvi_0_20171015.tif'
results = 'results/'

#Input your area of interest's name, coords, and 
#the year you're interested in?
AOI = 'Murrumbidgee'
year = 'Summer2017-18'

#What thresholds should I use?
threshold = 0.8
wofs_theshold = 0.15
#-----------------------------------------

In [ ]:
#Creating a folder to keep things neat
directory = results + AOI + "_" + year
if not os.path.exists(directory):
    os.mkdir(directory)

results = results + AOI + "_" + year + "/"

## load data

## Image Segmentation

In [ ]:
# setup input filename
InputNDVIStats = data
KEAFile = results + AOI + '_' + year + '.kea'
SegmentedKEAFile = results + AOI + '_' + year + '_sheperdSEG.kea'
SegmentedTiffFile = results + AOI + '_' + year + '_sheperdSEG.tif'
SegmentedPolygons = results + AOI + '_' + year + '_SEGpolygons.shp'
print("calculating imageSegmentation")
imageSeg(InputNDVIStats, KEAFile, SegmentedKEAFile, SegmentedTiffFile, SegmentedPolygons)

### Zonal Statistics & filtering

In [ ]:
InputNDVIStats = 'data/datacube_stats/summer_2004_2018/tifs/ndvi_0_20171015.tif'

In [ ]:
gdf = gpd.read_file(results + AOI + '_' + year + '_SEGpolygons.shp')
#calculate zonal mean of NDVI
gdf['mean'] = pd.DataFrame(zonal_stats(vectors=gdf['geometry'], raster=InputNDVIStats, stats='mean'))['mean']
#calculate area of polygons
gdf['area'] = gdf['geometry'].area
#filter by area and mean NDVI
highNDVI = gdf['mean'] >= threshold
smallArea = gdf['area'] <= 5500000
gdf = gdf[highNDVI & smallArea]
#export shapefile
gdf.to_file(results + AOI + "_" + year + "_Irrigated.shp")

## Masking

In [ ]:
#get the transform and projection of our gtiff
transform, projection = transform_tuple(NDVI_max, (NDVI_max.x, NDVI_max.y), epsg=3577)
#find the width and height of the xarray dataset we want to mask
width,height = NDVI_max.shape
# rasterize vector
gdf_raster = SpatialTools.rasterize_vector(results + AOI + "_" + year + "_Irrigated.shp",
                                           height, width, transform, projection, raster_path=None)
# Mask the xarray
NDVI_max_Irrigated = NDVI_max.where(gdf_raster)

In [1]:
pwd

'/g/data1a/u46/users/cb3058/ICE_project'

## Reclassify & raster math

In [ ]:
#remove areas below our threshold that are at the edges of the rasterized polygons
NDVI_max_Irrigated = NDVI_max_Irrigated.where(NDVI_max_Irrigated >= threshold)
#Use wofs to remove areas that have standing water for a significant amount of time
NDVI_max_Irrigated = NDVI_max_Irrigated.where(wofs_alltime.frequency.drop('time').squeeze() <= wofs_theshold)

#remove pixels that cross over the major rivers in the region
rivers_raster = SpatialTools.rasterize_vector("data/spatial/major_rivers_aus.shp", height, width, transform, projection, raster_path=None)
rivers_raster = rivers_raster.astype(bool)
xr.DataArray(rivers_raster, coords = [NDVI_max.y, NDVI_max.x], dims = ['y', 'x'], name='irrigated_areas')
NDVI_max_Irrigated = NDVI_max_Irrigated.where(rivers_raster == 0)

In [ ]:
#What is the area of irrigation?
# ones = np.count_nonzero(~np.isnan(NDVI_max_Irrigated.values))
# area = (ones*(25*25)) / 1000000
# print("Around " + AOI + " during " + str(year) + ", " + str(area) + " km2 was under irrigated cultivation")

## export results as GTiff

In [ ]:
SpatialTools.array_to_geotiff(results + AOI + "_" + year + "_Irrigated.tif",
              NDVI_max_Irrigated.values,
              geo_transform = transform, 
              projection = projection, 
              nodata_val=0)